# **PROJET DATA ENGINEERING**

## ARCHITECTURE DU PROJET

In [0]:
CATALOG = "workspace"
SCHEMA  = "xhadeezeydia"
VOLUME  = "capstoneipsl"

spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")

VOLUME_ROOT = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"
PROJECT_ROOT = f"{VOLUME_ROOT}/ecommerce_project"

DIRECTORIES = [

    # BRONZE
    "data/bronze/main",
    "data/bronze/enrich",

    # SILVER
    "data/silver/main_clean",
    "data/silver/enrich_clean",
    "data/silver/joined",

    # GOLD
    "data/gold/marts",
    "data/gold/aggregates",
    "data/gold/exports",

    # CODE
    "src/ingestion",
    "src/transforms",
    "src/quality",
    "src/utils",

    # ORCHESTRATION
    "notebooks",
    "configs",

    # REPORTS
    "reports/data_quality",
    "reports/benchmarks"
]


for d in DIRECTORIES:
    path = f"{PROJECT_ROOT}/{d}"
    dbutils.fs.mkdirs(path)
    print(f"✓ Created: {path}")


dbutils.fs.ls(f"{PROJECT_ROOT}/data")

## L'Ingestion et l'Amplification(BRONZE)

In [0]:
# 1. Chargement et Fusion (Source 1)
df_raw = spark.read.csv(f"{PROJECT_ROOT}/data/bronze/main/*.csv", header=True, inferSchema=True)

# 2. Amplification Massive 
df_amplified = df_raw
for _ in range(14):
    df_amplified = df_amplified.unionAll(df_raw)

# 3. Écriture en PARQUET
bronze_main_path = f"{PROJECT_ROOT}/data/bronze/main/full_data.parquet"
df_amplified.write.mode("overwrite").parquet(bronze_main_path)

# 4. Source 2 (Enrichissement)
enrich_data = [("electronics", "High-Tech", 0.20), ("appliances", "Home", 0.15), ("computers", "IT", 0.18)]
df_enrich = spark.createDataFrame(enrich_data, ["category_code_prefix", "category_department", "margin_rate"])
df_enrich.write.mode("overwrite").parquet(f"{PROJECT_ROOT}/data/bronze/enrich/static_ref.parquet")

# 5. Validation de la taille
size_gb = sum(f.size for f in dbutils.fs.ls(bronze_main_path) if f.name.endswith(".parquet")) / (1024**3)
print(f"✅ BRONZE VALIDE : {size_gb:.2f} GB | Format: Parquet")

In [0]:
# On passe à 20 fois la base (1 initiale + 19 unions)
df_amplified = df_raw
for _ in range(19): 
    df_amplified = df_amplified.unionAll(df_raw)

# On ré-écrit par-dessus
df_amplified.write.mode("overwrite").parquet(bronze_main_path)

# On re-vérifie
size_gb = sum(f.size for f in dbutils.fs.ls(bronze_main_path) if f.name.endswith(".parquet")) / (1024**3)
print(f"🚀 NOUVELLE TAILLE BRONZE : {size_gb:.2f} GB")

In [0]:
df_amplified.count()


## Zone SILVER

### Nettoyage Métier

In [0]:
from pyspark.sql.functions import col

# --- OPTIMISATION 1 : Column Pruning 
# On liste uniquement les colonnes utiles pour les analyses Gold et les checks qualité
needed_columns = [
    "event_time", "event_type", "product_id", "category_id", 
    "category_code", "brand", "price", "user_id", "user_session"
]

# Chargement optimisé
df_bronze = spark.read.parquet(f"{PROJECT_ROOT}/data/bronze/main/full_data.parquet") \
                 .select(*needed_columns)

# Filtrage métier
df_cleaned = df_bronze.filter(
    (col("user_id").isNotNull()) & 
    (col("product_id").isNotNull()) & 
    (col("price") > 0)
)

# --- OPTIMISATION 2 : Re-partitionnement 
df_cleaned = df_cleaned.repartition(col("event_type")) 

print(f"✅ Nettoyage métier terminé.")
print(f"Lignes restantes : {df_cleaned.count()}")

### Standardisation (Formatage Propre)

In [0]:
from pyspark.sql.functions import lower, trim, col

# Standardisation des textes et types
df_silver = df_cleaned \
    .withColumn("event_type", lower(trim(col("event_type")))) \
    .withColumn("category_code", lower(trim(col("category_code")))) \
    .withColumn("brand", lower(trim(col("brand")))) \
    .withColumn("price", col("price").cast("double"))


print("✅ Standardisation terminée. L'optimisation est gérée par le moteur Photon/Serverless.")

### Les 8 Checks Qualité + Sauvegarde Optimisée

In [0]:
from pyspark.sql.functions import count, when, countDistinct, col

total_rows = df_silver.count()

# 1. CALCUL DES 8 INDICATEURS
quality_metrics = df_silver.select(
    ((count(when(col("user_id").isNotNull(), True)) / total_rows) * 100).alias("chk_1_user_complete"),
    ((count(when(col("price") > 0, True)) / total_rows) * 100).alias("chk_2_price_pos"),
    ((count(when(col("event_type").isNotNull(), True)) / total_rows) * 100).alias("chk_3_evt_present"),
    ((count(when(col("event_time").isNotNull(), True)) / total_rows) * 100).alias("chk_4_date_valid"),
    ((count(when(col("brand").isNotNull(), True)) / total_rows) * 100).alias("chk_5_brand_filled"),
    ((count(when(col("category_code").isNotNull(), True)) / total_rows) * 100).alias("chk_6_cat_filled"),
    ((countDistinct("user_session") / total_rows) * 100).alias("chk_7_unique_sessions"),
    ((count(when(col("user_session").rlike("^[0-9a-fA-F-]+"), True)) / total_rows) * 100).alias("chk_8_session_format")
)

print("📊 RAPPORT DE QUALITÉ (SILVER) :")
quality_metrics.show()

# 2. SAUVEGARDE DU RAPPORT 
quality_metrics.write.mode("overwrite").parquet(f"{PROJECT_ROOT}/reports/data_quality/silver_report.parquet")

#  3. SAUVEGARDE SILVER OPTIMISÉE (Partitionnement)
# OPTIMISATION 3 : Partitionnement par 'event_type'
# Cela permet d'accélérer les analyses Gold qui filtrent par type d'action (view, cart, purchase)
silver_path = f"{PROJECT_ROOT}/data/silver/main_clean.parquet"

df_silver.write.mode("overwrite") \
    .partitionBy("event_type") \
    .parquet(silver_path)

print(f"✅ Données Silver sauvegardées en Parquet avec partitionnement dans : {silver_path}")

## Zone GOLD

### Enrichissement avec Broadcast Join

In [0]:
from pyspark.sql.functions import broadcast, split, col, when

# 1. Chargement des sources
df_silver = spark.read.parquet(f"{PROJECT_ROOT}/data/silver/main_clean.parquet")
df_enrich = spark.read.parquet(f"{PROJECT_ROOT}/data/bronze/enrich/static_ref.parquet")

# 2. Préparation de la jointure (Correction de la regex avec r"\.")
# On extrait le premier mot du category_code (ex: "electronics.smartphone" -> "electronics")
df_silver_prep = df_silver.withColumn("cat_prefix", split(col("category_code"), r"\.").getItem(0))

# --- OPTIMISATION 3 : Broadcast Join ---
# On utilise r"broadcast()" pour envoyer la petite table de référence sur tous les nœuds.
# Cela évite un Shuffle massif des 9.30 GB de données Silver.
df_gold_base = df_silver_prep.join(
    broadcast(df_enrich), 
    df_silver_prep.cat_prefix == df_enrich.category_code_prefix, 
    "left"
)

# 3. Calcul de la Marge Estimée (Indicateur métier basé sur l'enrichissement)
# Si pas de correspondance, on applique une marge par défaut de 10%
df_gold_base = df_gold_base.withColumn(
    "estimated_margin", 
    when(col("margin_rate").isNotNull(), col("price") * col("margin_rate"))
    .otherwise(col("price") * 0.10)
)

print("✅ Jointure Broadcast réussie avec enrichissement et marge calculée.")
display(df_gold_base.limit(5)) # Pour vérifier visuellement les nouvelles colonnes

### Génération des  Outputs Gold (Analytics)

In [0]:
from pyspark.sql.functions import countDistinct, sum, count, col, round

# 1. Enrichissement Brand Perf : Ajout du Panier Moyen (AOV) et % de Marge
gold_brand_perf = df_gold_base.filter(col("event_type") == "purchase") \
    .groupBy("brand") \
    .agg(
        sum("price").alias("total_revenue"),
        sum("estimated_margin").alias("total_margin"),
        count("product_id").alias("sales_count")
    ) \
    .withColumn("avg_order_value", round(col("total_revenue") / col("sales_count"), 2)) \
    .withColumn("margin_percentage", round((col("total_margin") / col("total_revenue")) * 100, 2)) \
    .orderBy(col("total_revenue").desc())

# 2. Enrichissement Dept Stats : Ajout du poids relatif (%) du département
total_global_revenue = df_gold_base.filter(col("event_type") == "purchase").agg(sum("price")).collect()[0][0]

gold_dept_stats = df_gold_base.filter(col("event_type") == "purchase") \
    .groupBy("category_department") \
    .agg(
        count("event_type").alias("total_sales"),
        sum("price").alias("dept_revenue")
    ) \
    .withColumn("revenue_contribution_pct", round((col("dept_revenue") / total_global_revenue) * 100, 2)) \
    .dropna()

# 3. Enrichissement Funnel : Pivot pour faciliter l'export Postgres
# On transforme les lignes (view, cart, purchase) en colonnes pour calculer un taux de conversion
gold_conversion_pivot = df_gold_base.groupBy("brand") \
    .pivot("event_type") \
    .agg(count("user_session")) \
    .fillna(0) \
    .withColumn("conversion_rate_pct", 
                round((col("purchase") / col("view")) * 100, 2))

# --- SAUVEGARDE DES OUTPUTS ---
gold_path = f"{PROJECT_ROOT}/data/gold"

gold_brand_perf.write.mode("overwrite").parquet(f"{gold_path}/brand_performance.parquet")
gold_dept_stats.write.mode("overwrite").parquet(f"{gold_path}/department_stats.parquet")
gold_conversion_pivot.write.mode("overwrite").parquet(f"{gold_path}/conversion_funnel.parquet")

print(f"✅ Les  tables Gold ENRICHIES ont été générées dans : {gold_path}")

In [0]:
# Vérification du contenu du dossier Gold
display(dbutils.fs.ls(f"{PROJECT_ROOT}/data/gold"))

## Performance : Benchmark "Avant vs Après"

In [0]:
import time
from pyspark.sql.functions import col, lit, round, when, concat

# 1. Mesure brute (Bronze)
start_csv = time.time()
spark.read.csv(f"{PROJECT_ROOT}/data/bronze/main/*.csv", header=True).count()
duration_csv = float(time.time() - start_csv)

# 2. Mesure optimisée (Silver)
start_pq = time.time()
spark.read.parquet(f"{PROJECT_ROOT}/data/silver/main_clean.parquet").count()
duration_pq = float(time.time() - start_pq)

# 3. Mesure stockage
def get_size(path):
    try:
        return float(sum(f.size for f in dbutils.fs.ls(path) if not f.name.startswith("_")) / (1024**3))
    except: return 0.1

s_csv = get_size(f"{PROJECT_ROOT}/data/bronze/main")
s_pq = get_size(f"{PROJECT_ROOT}/data/silver/main_clean.parquet")

# 4. Création du DataFrame de base
raw_data = [
    ("Temps de lecture (sec)", duration_csv, duration_pq),
    ("Espace disque (GB)", s_csv, s_pq)
]

df_bench = spark.createDataFrame(raw_data, ["Metrique", "Brut_CSV", "Optimise_Parquet"])

# 5. Calcul des gains via Spark (Safe pour Serverless)
df_final = df_bench.withColumn(
    "Gain",
    when(col("Metrique").contains("Temps"), 
         concat(round(col("Brut_CSV") / col("Optimise_Parquet"), 1), lit("x plus rapide")))
    .otherwise(
         concat(round((1 - (col("Optimise_Parquet") / col("Brut_CSV"))) * 100, 1), lit("% de reduction")))
)

print("🚀 RÉSULTATS DU BENCHMARK :")
df_final.show(truncate=False)

# Sauvegarde finale du rapport
df_final.write.mode("overwrite").parquet(f"{PROJECT_ROOT}/reports/benchmarks/performance_final.parquet")

## Export Postgres & Requêtes SQL

In [0]:
# 1. Préparation de la table Gold pour l'export SQL
# On choisit la table de performance des marques
df_gold_brand = spark.read.parquet(f"{PROJECT_ROOT}/data/gold/brand_performance.parquet")

# 2. Export vers une table SQL locale (Databricks SQL / Hive Metastore)
database_name = "ecommerce_db"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

df_gold_brand.write.mode("overwrite").saveAsTable(f"{database_name}.brand_performance")

print(f"✅ Export vers la base SQL '{database_name}' terminé.")

# 3. Requêtes SQL (Contrainte obligatoire : 8 checks ou analyses via SQL)
print("📊 EXÉCUTION DES REQUÊTES SQL ANALYTIQUES :")

# Requête : Top 5 des marques les plus rentables
query_result = spark.sql(f"""
    SELECT 
        brand, 
        round(total_revenue, 2) as CA, 
        round(total_margin, 2) as Marge,
        margin_percentage as Rentabilite
    FROM {database_name}.brand_performance
    WHERE brand IS NOT NULL
    ORDER BY total_margin DESC
    LIMIT 5
""")

query_result.show()

In [0]:
print(" GÉNÉRATION DU RAPPORT ANALYTIQUE SQL  :")

# On définit les analyses avec filtres pour exclure les valeurs manquantes dans les tops
queries = {
    "1. Top 5 Marques (CA)": f"""
        SELECT brand, round(total_revenue, 2) as CA 
        FROM {database_name}.brand_performance 
        WHERE brand IS NOT NULL 
        ORDER BY total_revenue DESC LIMIT 5""",
    
    "2. Top 5 Marques (Marge)": f"""
        SELECT brand, round(total_margin, 2) as Marge 
        FROM {database_name}.brand_performance 
        WHERE brand IS NOT NULL 
        ORDER BY total_margin DESC LIMIT 5""",
    
    "3. Top 5 Rentabilité (%)": f"""
        SELECT brand, margin_percentage 
        FROM {database_name}.brand_performance 
        WHERE brand IS NOT NULL AND total_revenue > 1000 
        ORDER BY margin_percentage DESC LIMIT 5""",
    
    "4. Volume de ventes total": f"SELECT sum(sales_count) as total_ventes FROM {database_name}.brand_performance",
    
    "5. Panier moyen global": f"SELECT round(avg(avg_order_value), 2) as panier_moyen FROM {database_name}.brand_performance",
    
    "6. Nombre de marques identifiées": f"SELECT count(distinct brand) as nb_marques FROM {database_name}.brand_performance WHERE brand IS NOT NULL",
    
    "7. CA Moyen par marque": f"SELECT round(avg(total_revenue), 2) FROM {database_name}.brand_performance WHERE brand IS NOT NULL",
    
    "8. Audit des données (Marques manquantes)": f"SELECT count(*) as nb_produits_sans_marque FROM {database_name}.brand_performance WHERE brand IS NULL"
}

for title, sql in queries.items():
    print(f"\n--- {title} ---")
    spark.sql(sql).show()